In [6]:
import pickle 
import torch 
import os 
import sys 
sys.path.append('../../../submodule') 
import torch_util 
from torch_util.imports import * 

with open(os.path.expanduser('~/dataset/HGB/processed/ACM_hg.dict.pkl'), 'rb') as fp: 
    graph_info = pickle.load(fp)
    
graph_info 

{'feat_dict': {'author': array([[1., 1., 1., ..., 0., 0., 0.],
         [1., 1., 1., ..., 0., 0., 0.],
         [1., 1., 1., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
  'paper': array([[1., 1., 1., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
  'subject': array([[1., 1., 1., ..., 0., 0., 0.],
         [1., 1., 1., ..., 1., 1., 1.],
         [1., 1., 1., ..., 1., 0., 1.],
         ...,
         [0., 1., 0., ..., 0., 0., 0.],
         [0., 1., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)},
 'num_nodes_dict': {'author': 5959,
  'paper': 3025,
  'subject': 56,
  'term': 1902},
 'edge_index_dict': {('paper',
   'pp',
   'paper'): arr

In [7]:
edge_index_dict = graph_info['edge_index_dict']
num_nodes_dict = graph_info['num_nodes_dict']

{ k: v.shape for k, v in edge_index_dict.items() }

{('paper', 'pp', 'paper'): (2, 5343),
 ('paper', 'pp_rev', 'paper'): (2, 5343),
 ('paper', 'pa', 'author'): (2, 9949),
 ('author', 'ap', 'paper'): (2, 9949),
 ('paper', 'pt', 'term'): (2, 255619),
 ('term', 'tp', 'paper'): (2, 255619),
 ('paper', 'ps', 'subject'): (2, 3025),
 ('subject', 'sp', 'paper'): (2, 3025)}

In [8]:
os.makedirs('./TransE_emb', exist_ok=True)

def convert_hg_to_triplets(edge_index_dict: dict[EdgeType, EdgeIndex], 
                           num_nodes_dict: dict[NodeType, int],
                           output_path: str): 
    nid_offset_dict: dict[NodeType, int] = dict() 

    nid_offset = 0 

    for ntype, num_nodes in sorted(num_nodes_dict.items()):
        nid_offset_dict[ntype] = nid_offset 
        nid_offset += num_nodes 

    with open(output_path, 'w', encoding='utf-8') as fp:
        for etype, edge_index in tqdm(edge_index_dict.items()): 
            src_ntype, _, dest_ntype = etype 
            etype = '__'.join(etype) 
            edge_index = torch.tensor(edge_index, dtype=torch.int64)
            src_nids, dest_nids = edge_index 
            
            src_nids.add_(nid_offset_dict[src_ntype])
            dest_nids.add_(nid_offset_dict[dest_ntype])

            for src_nid, dest_nid in zip(src_nids.tolist(), dest_nids.tolist()): 
                print(f"{src_nid}\t{etype}\t{dest_nid}", file=fp)

convert_hg_to_triplets(
    edge_index_dict = edge_index_dict, 
    num_nodes_dict = num_nodes_dict,
    output_path = './TransE_emb/triplets.tsv', 
)

  0%|          | 0/8 [00:00<?, ?it/s]